<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/main/QnA_Llama_3_1_8b_with_Unsloth_D1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

fatal: destination path 'qM-AI-L' already exists and is not an empty directory.


In [151]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [152]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

# **Setting Up the Environment**

In [153]:
!pip install rouge_score
from rouge_score import rouge_scorer

In [154]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import torch
import tensorflow as tf
import numpy as np
import time
from google.colab import files
import shutil


In [155]:
import warnings
warnings.filterwarnings("ignore")

In [156]:
from torch import __version__
from packaging.version import Version as V
if V(__version__) < V("2.4.0"):
  xformers = "xformers==0.0.27"
else:
  xformers =  "xformers"

In [157]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer

In [158]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# **Downloading 4-bit Quantized Model and Adding LoRA Adapters**

# **Dataset-1 :Preparing the Dataset for Fine-tuning**

In [159]:
D1_train_path = '/content/qM-AI-L/QA-Chatbot/Dataset-1/aiml-qa-train.xlsx'
D1_test_path = '/content/qM-AI-L/QA-Chatbot/Dataset-1/aiml-qa-test.xlsx'
D1_dev_path = '/content/qM-AI-L/QA-Chatbot/Dataset-1/aiml-qa-dev.xlsx'

In [160]:
D1_train_df = pd.read_excel(D1_train_path)
D1_dev_df = pd.read_excel(D1_dev_path)
D1_test_df = pd.read_excel(D1_test_path)

In [161]:
D1_train_df.shape, D1_test_df.shape, D1_dev_df.shape

((1316, 2), (120, 3), (80, 3))

In [162]:
D1_train_df.columns,  D1_test_df.columns,  D1_dev_df.columns

(Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer1', 'answer2'], dtype='object'),
 Index(['question', 'answer1', 'answer2'], dtype='object'))

In [163]:
# change column names in D1 df to match with test and dev dfs

D1_train_df = D1_train_df.rename(columns={"answer": "answer1"})


In [164]:
D1_train_df.columns,  D1_test_df.columns,  D1_dev_df.columns

(Index(['question', 'answer1'], dtype='object'),
 Index(['question', 'answer1', 'answer2'], dtype='object'),
 Index(['question', 'answer1', 'answer2'], dtype='object'))

## **Add prompt format column to train df for model input**

In [165]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "answer for this question", # instruction
        "Is 9.11 larger than 9.9?", # input
        "", # output—leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
answer for this question

### Input:
Is 9.11 larger than 9.9?

### Response:
No, 9.11 is not larger than 9.9 because 9.11 is less than 9.9.
<|end_of_text|>


In [166]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<|end_of_text|>'

In [167]:
BOS_TOKEN = tokenizer.bos_token
BOS_TOKEN

'<|begin_of_text|>'

In [168]:
#Function for Train dataset
EOS_TOKEN = tokenizer.eos_token
def formatted_train(x):

  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
    {x['answer1']} {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
    {x['answer1']} {EOS_TOKEN}"""

  return formatted_text

In [169]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatted_train(x):
#     instructions = "Generate an answer for the following question in less than three sentences."
#     inputs       = x['question']
#     outputs      = x['answer1']
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }

In [170]:
#Function for Test and Dev dataset
def format_test(x):
  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
     {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
     {EOS_TOKEN}"""

  return formatted_text

In [171]:
D1_train_df

,question,answer1
0,What is the difference between concatenation v...,Concatenation combines two tensors by adding t...
1,What is the difference between concatenation v...,Concatenation is often used to combine differe...
2,Why are derivatives substracted from weights?,The derivative of the loss function at a point...
3,Why are derivatives substracted from weights?,The intuition behind adjusting the weights by ...
4,Describe a process/pipeline for generating rep...,Select a pretrained model suitable for the tas...
...,...,...
1311,Where fully connected layer is used?,It's used in the final layers of neural networ...
1312,What are the examples of high dimension data i...,High-dimensional data have become more common ...
1313,What are the examples of high dimension data i...,High-dimensional datasets have a large number ...
1314,How do we split data sets between DEV and TEST,"Typically, data sets are split into a developm..."


In [172]:
D1_train_df["formatted_text"] = D1_train_df.apply(formatted_train, axis=1)
D1_test_df["formatted_text"] = D1_test_df.apply(format_test, axis=1)
D1_dev_df["formatted_text"] = D1_dev_df.apply(format_test, axis=1)

**Check train prompt includes answer column/ data, and test and dev prompts does not include answers/targets**

In [173]:
D1_train_df

,question,answer1,formatted_text
0,What is the difference between concatenation v...,Concatenation combines two tensors by adding t...,<|begin_of_text|> Below is an instruction tha...
1,What is the difference between concatenation v...,Concatenation is often used to combine differe...,<|begin_of_text|> Below is an instruction tha...
2,Why are derivatives substracted from weights?,The derivative of the loss function at a point...,<|begin_of_text|> Below is an instruction tha...
3,Why are derivatives substracted from weights?,The intuition behind adjusting the weights by ...,<|begin_of_text|> Below is an instruction tha...
4,Describe a process/pipeline for generating rep...,Select a pretrained model suitable for the tas...,<|begin_of_text|> Below is an instruction tha...
...,...,...,...
1311,Where fully connected layer is used?,It's used in the final layers of neural networ...,<|begin_of_text|> Below is an instruction tha...
1312,What are the examples of high dimension data i...,High-dimensional data have become more common ...,<|begin_of_text|> Below is an instruction tha...
1313,What are the examples of high dimension data i...,High-dimensional datasets have a large number ...,<|begin_of_text|> Below is an instruction tha...
1314,How do we split data sets between DEV and TEST,"Typically, data sets are split into a developm...",<|begin_of_text|> Below is an instruction tha...


In [174]:
D1_train_df['formatted_text'][1]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is the difference between concatenation vs. summation of two tensors?\n\n    ### Response:\n    Concatenation is often used to combine different features of a data set, such as images and text. Summation is often used to calculate the similarity between two tensors, or to compute the loss function for a neural network. <|end_of_text|>'

In [175]:
D1_test_df['formatted_text'][0]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    How we can effectively convert 2D images to 1D?\n\n    ### Response:\n     <|end_of_text|>'

In [176]:
D1_dev_df['formatted_text'][0]

" <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    Does the maximum value of 'n' in the n-gram model depend on the average number of words per sentence in the training data?\n\n    ### Response:\n     <|end_of_text|>"

# **Convert DFs to Datasets**

In [177]:
!pip install datasets

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [179]:
from datasets import Dataset
D1_Train_dataset = Dataset.from_pandas(D1_train_df)
D1_Test_dataset = Dataset.from_pandas(D1_test_df)
D1_Dev_dataset = Dataset.from_pandas(D1_dev_df)

In [180]:
D1_Train_dataset, D1_Test_dataset, D1_Dev_dataset

(Dataset({
     features: ['question', 'answer1', 'formatted_text'],
     num_rows: 1316
 }),
 Dataset({
     features: ['question', 'answer1', 'answer2', 'formatted_text'],
     num_rows: 120
 }),
 Dataset({
     features: ['question', 'answer1', 'answer2', 'formatted_text'],
     num_rows: 80
 }))

In [181]:
#check train dataset
D1_Train_dataset['formatted_text'][0:2]

[' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is the difference between concatenation vs. summation of two tensors?\n\n    ### Response:\n    Concatenation combines two tensors by adding them together along a specified dimension. Summation adds the elements of two tensors together element-wise. <|end_of_text|>',
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is the difference between concatenation vs. summation of two tenso

# **Fine-tuning Google Gemma for Code Dataset**


In [182]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [183]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [184]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = D1_Train_dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = 1024,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1316 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [185]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,316 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.146000
2,3.196000
3,3.201800
4,2.888400
5,2.653400
6,2.338300
7,2.032800
8,1.595500
9,1.374300
10,1.103800


**#check test data for infrencing and answer generation**

In [186]:
from transformers import TextStreamer

In [187]:
D1_test_df['formatted_text'][2], D1_test_df['formatted_text'][3]

(" <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is NLP's current biggest challenge that is being tried to overcome ?\n\n    ### Response:\n     <|end_of_text|>",
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    Which problems cannot be solved by Neural networks?\n\n    ### Response:\n     <|end_of_text|>')

# **Checkout the trained model generating answer**

In [188]:
Prompt = D1_test_df['formatted_text'][45]


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
# Get the generated text by decoding the IDs
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text


<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How we can apply CNN to text and pictures?

    ### Response:
     <|end_of_text|><|begin_of_text|> <|begin_of_text|>  Convolutional Neural Networks (CNNs) can be applied to both text and pictures by leveraging convolutional layers that extract features from the input data. <|end_of_text|>


# **Function to extract answer part from the model generated response**

In [189]:
generated_text

'  Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    How we can apply CNN to text and pictures?\n\n    ### Response:\n        Convolutional Neural Networks (CNNs) can be applied to both text and pictures by leveraging convolutional layers that extract features from the input data. '

In [215]:

# **Function to extract answer part from the model generated response**
def get_answer(text):
    start_tag = "### Response:"

    # Find the start and end indices
    start_idx = text.find(start_tag)

    # Check if both tags are found
    if start_idx == -1:
        return None  # Tags not found

    # Extract content between the tags
    answer = text[start_idx + len(start_tag):].strip()

    return answer

get_answer(generated_text)

'Convolutional Neural Networks (CNNs) can be applied to both text and pictures by leveraging convolutional layers that extract features from the input data.'

In [216]:
D1_test_df.columns, D1_test_df.shape

(Index(['question', 'answer1', 'answer2', 'formatted_text'], dtype='object'),
 (120, 4))

In [217]:
original_answers = []
llama_answers = []

for index, row in D1_test_df.iterrows():
    original_answer = row['answer1']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D1_Test_Gemma_with_Unsloth_df = pd.DataFrame(zipped_subjects, columns = ['original_answers', 'Llama_answers'])
D1_Test_Gemma_with_Unsloth_df

<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How we can effectively convert 2D images to 1D?

    ### Response:
     <|end_of_text|><|begin_of_text|>://We can effectively convert 2D images to 1D by flattening the image into a single array. This involves reshaping the image by flattening the rows and columns into a single dimension, resulting in a 1D representation. <|end_of_text|>
<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ##

,original_answers,Llama_answers
0,Converting images to 1D data may not be effect...,://We can effectively convert 2D images to 1D ...
1,"Yes, autoencoders can be applied to numerical ...",":// Yes, autoencoders can be used for dimensio..."
2,The main challenges of NLP is finding and coll...,:// NLP's biggest challenge is to create mo...
3,While neural networks have shown great success...,"No, neural networks can solve a wide range of ..."
4,"Yes, scaling the input data is generally recom...",SVM is sensitive to the scale of the data. Sca...
...,...,...
115,Data mining refers to the process of discoveri...,Data mining is the process of discovering patt...
116,"CLAMP (Clinical Language Annotation, Modeling,...","://Yes, there are several software tools avail..."
117,Slicing is a useful technique in Python for ex...,When we need to extract a portion of data or f...
118,"Yes, lemmatization is generally considered bet...",:// Lemmatization is generally considered supe...


In [218]:
# prompt: Using dataframe D1_Test_Gemma_with_Unsloth_df: get all rows which that are not None in Llama_answers

# Select rows where Llama_answers is not null
D1_Test_Llama3_with_Unsloth = D1_Test_Gemma_with_Unsloth_df[D1_Test_Gemma_with_Unsloth_df['Llama_answers'].notna()].reset_index()
D1_Test_Llama3_with_Unsloth


,index,original_answers,Llama_answers
0,0,Converting images to 1D data may not be effect...,://We can effectively convert 2D images to 1D ...
1,1,"Yes, autoencoders can be applied to numerical ...",":// Yes, autoencoders can be used for dimensio..."
2,2,The main challenges of NLP is finding and coll...,:// NLP's biggest challenge is to create mo...
3,3,While neural networks have shown great success...,"No, neural networks can solve a wide range of ..."
4,4,"Yes, scaling the input data is generally recom...",SVM is sensitive to the scale of the data. Sca...
...,...,...,...
115,115,Data mining refers to the process of discoveri...,Data mining is the process of discovering patt...
116,116,"CLAMP (Clinical Language Annotation, Modeling,...","://Yes, there are several software tools avail..."
117,117,Slicing is a useful technique in Python for ex...,When we need to extract a portion of data or f...
118,118,"Yes, lemmatization is generally considered bet...",:// Lemmatization is generally considered supe...


# ** Evaluate model performance- Rouge**

In [219]:
!pip install rouge-score  # Install the rouge-score library
from rouge_score import rouge_scorer

In [220]:
D1_test_scores = D1_Test_Llama3_with_Unsloth.copy()

In [221]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True) # Instantiate a RougeScorer object
    scores = scorer.score(reference, hypothesis) # Call the score method on the RougeScorer object
    return {metric: score.fmeasure for metric, score in scores.items()}

# Apply ROUGE score calculation to each row
D1_test_scores['rouge_scores'] = D1_test_scores.apply(lambda row: calculate_rouge(row['original_answers'], row['Llama_answers']), axis=1)

In [222]:
D1_test_scores

,index,original_answers,Llama_answers,rouge_scores
0,0,Converting images to 1D data may not be effect...,://We can effectively convert 2D images to 1D ...,"{'rouge1': 0.28571428571428575, 'rouge2': 0.06..."
1,1,"Yes, autoencoders can be applied to numerical ...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.608695652173913, 'rouge2': 0.3636..."
2,2,The main challenges of NLP is finding and coll...,:// NLP's biggest challenge is to create mo...,"{'rouge1': 0.24, 'rouge2': 0.0, 'rougeL': 0.15..."
3,3,While neural networks have shown great success...,"No, neural networks can solve a wide range of ...","{'rouge1': 0.21052631578947364, 'rouge2': 0.11..."
4,4,"Yes, scaling the input data is generally recom...",SVM is sensitive to the scale of the data. Sca...,"{'rouge1': 0.48148148148148145, 'rouge2': 0.19..."
...,...,...,...,...
115,115,Data mining refers to the process of discoveri...,Data mining is the process of discovering patt...,"{'rouge1': 0.5000000000000001, 'rouge2': 0.304..."
116,116,"CLAMP (Clinical Language Annotation, Modeling,...","://Yes, there are several software tools avail...","{'rouge1': 0.27999999999999997, 'rouge2': 0.12..."
117,117,Slicing is a useful technique in Python for ex...,When we need to extract a portion of data or f...,"{'rouge1': 0.3157894736842105, 'rouge2': 0.111..."
118,118,"Yes, lemmatization is generally considered bet...",:// Lemmatization is generally considered supe...,"{'rouge1': 0.4324324324324324, 'rouge2': 0.228..."


In [223]:
pd.json_normalize(D1_test_scores['rouge_scores'])

,rouge1,rouge2,rougeL,rougeLsum
0,0.285714,0.065574,0.222222,0.222222
1,0.608696,0.363636,0.478261,0.478261
2,0.240000,0.000000,0.160000,0.160000
3,0.210526,0.111111,0.210526,0.210526
4,0.481481,0.192308,0.333333,0.333333
...,...,...,...,...
115,0.500000,0.304348,0.458333,0.458333
116,0.280000,0.125000,0.200000,0.200000
117,0.315789,0.111111,0.315789,0.315789
118,0.432432,0.228571,0.378378,0.378378


In [224]:
D1_test_scores = pd.concat([D1_test_scores, pd.json_normalize(D1_test_scores['rouge_scores'])], axis=1)


In [225]:
D1_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,0,Converting images to 1D data may not be effect...,://We can effectively convert 2D images to 1D ...,"{'rouge1': 0.28571428571428575, 'rouge2': 0.06...",0.285714,0.065574,0.222222,0.222222
1,1,"Yes, autoencoders can be applied to numerical ...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.608695652173913, 'rouge2': 0.3636...",0.608696,0.363636,0.478261,0.478261
2,2,The main challenges of NLP is finding and coll...,:// NLP's biggest challenge is to create mo...,"{'rouge1': 0.24, 'rouge2': 0.0, 'rougeL': 0.15...",0.240000,0.000000,0.160000,0.160000
3,3,While neural networks have shown great success...,"No, neural networks can solve a wide range of ...","{'rouge1': 0.21052631578947364, 'rouge2': 0.11...",0.210526,0.111111,0.210526,0.210526
4,4,"Yes, scaling the input data is generally recom...",SVM is sensitive to the scale of the data. Sca...,"{'rouge1': 0.48148148148148145, 'rouge2': 0.19...",0.481481,0.192308,0.333333,0.333333
...,...,...,...,...,...,...,...,...
115,115,Data mining refers to the process of discoveri...,Data mining is the process of discovering patt...,"{'rouge1': 0.5000000000000001, 'rouge2': 0.304...",0.500000,0.304348,0.458333,0.458333
116,116,"CLAMP (Clinical Language Annotation, Modeling,...","://Yes, there are several software tools avail...","{'rouge1': 0.27999999999999997, 'rouge2': 0.12...",0.280000,0.125000,0.200000,0.200000
117,117,Slicing is a useful technique in Python for ex...,When we need to extract a portion of data or f...,"{'rouge1': 0.3157894736842105, 'rouge2': 0.111...",0.315789,0.111111,0.315789,0.315789
118,118,"Yes, lemmatization is generally considered bet...",:// Lemmatization is generally considered supe...,"{'rouge1': 0.4324324324324324, 'rouge2': 0.228...",0.432432,0.228571,0.378378,0.378378


In [226]:
!pip install nltk sacrebleu

In [227]:
import sacrebleu
from nltk.translate import meteor_score
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [228]:
# Tokenization function
def tokenize(text):
    return text.lower().split()


In [229]:
def calculate_bleu(reference, hypothesis):
    # BLEU requires tokenized input for reference, but not for hypothesis
    reference = [tokenize(reference)]
    bleu = sacrebleu.corpus_bleu([hypothesis], reference) # hypothesis should be a string
    return bleu.score


D1_test_scores['bleu_score'] = D1_test_scores.apply(lambda row: calculate_bleu(row['original_answers'], row['Llama_answers']), axis=1)

# Calculate METEOR score
def calculate_meteor(reference, hypothesis):
    # Tokenize both reference and hypothesis
    tokenized_reference = tokenize(reference)
    tokenized_hypothesis = tokenize(hypothesis)
    return meteor_score.single_meteor_score(tokenized_reference, tokenized_hypothesis)  # Pass tokenized inputs

D1_test_scores['meteor_score'] = D1_test_scores.apply(lambda row: calculate_meteor(row['original_answers'], row['Llama_answers']), axis=1)

In [230]:
D1_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,0,Converting images to 1D data may not be effect...,://We can effectively convert 2D images to 1D ...,"{'rouge1': 0.28571428571428575, 'rouge2': 0.06...",0.285714,0.065574,0.222222,0.222222,0.000000,0.143369
1,1,"Yes, autoencoders can be applied to numerical ...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.608695652173913, 'rouge2': 0.3636...",0.608696,0.363636,0.478261,0.478261,1.727224,0.454978
2,2,The main challenges of NLP is finding and coll...,:// NLP's biggest challenge is to create mo...,"{'rouge1': 0.24, 'rouge2': 0.0, 'rougeL': 0.15...",0.240000,0.000000,0.160000,0.160000,0.000000,0.080128
3,3,While neural networks have shown great success...,"No, neural networks can solve a wide range of ...","{'rouge1': 0.21052631578947364, 'rouge2': 0.11...",0.210526,0.111111,0.210526,0.210526,0.000000,0.097540
4,4,"Yes, scaling the input data is generally recom...",SVM is sensitive to the scale of the data. Sca...,"{'rouge1': 0.48148148148148145, 'rouge2': 0.19...",0.481481,0.192308,0.333333,0.333333,0.000000,0.232027
...,...,...,...,...,...,...,...,...,...,...
115,115,Data mining refers to the process of discoveri...,Data mining is the process of discovering patt...,"{'rouge1': 0.5000000000000001, 'rouge2': 0.304...",0.500000,0.304348,0.458333,0.458333,1.055972,0.378274
116,116,"CLAMP (Clinical Language Annotation, Modeling,...","://Yes, there are several software tools avail...","{'rouge1': 0.27999999999999997, 'rouge2': 0.12...",0.280000,0.125000,0.200000,0.200000,0.000000,0.116732
117,117,Slicing is a useful technique in Python for ex...,When we need to extract a portion of data or f...,"{'rouge1': 0.3157894736842105, 'rouge2': 0.111...",0.315789,0.111111,0.315789,0.315789,0.000000,0.269006
118,118,"Yes, lemmatization is generally considered bet...",:// Lemmatization is generally considered supe...,"{'rouge1': 0.4324324324324324, 'rouge2': 0.228...",0.432432,0.228571,0.378378,0.378378,1.335353,0.423129


In [231]:
average_scores = {
    'average_rouge1': D1_test_scores['rouge1'].mean(),
    'average_rouge2': D1_test_scores['rouge2'].mean(),
    'average_rougeL': D1_test_scores['rougeL'].mean(),
    'average_rougeLsum': D1_test_scores['rougeLsum'].mean(),
    'average_bleu_score': D1_test_scores['bleu_score'].mean(),
    'average_meteor_score': D1_test_scores['meteor_score'].mean(),
}

In [232]:
print("Average Scores for Dataset-1 (Test-Answer-1) QnA task using Gemma_7b_with_Unsloth model:\n")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")

Average Scores for Dataset-1 (Test-Answer-1) QnA task using Gemma_7b_with_Unsloth model:

average_rouge1: 0.4005
average_rouge2: 0.1773
average_rougeL: 0.3252
average_rougeLsum: 0.3281
average_bleu_score: 0.6938
average_meteor_score: 0.2682


# **Repeat above evaluation process for the 2nd Answer annotation**

In [234]:
original_answers = []
llama_answers = []

for index, row in D1_test_df.iterrows():
    original_answer = row['answer2']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D1_Test_llama_with_Unsloth_df_A2 = pd.DataFrame(zipped_subjects, columns = ['answer-2', 'Llama_answers'])
D1_Test_llama_with_Unsloth_df_A2

<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How we can effectively convert 2D images to 1D?

    ### Response:
     <|end_of_text|><|begin_of_text|>://We can use different methods to convert 2D images to 1D, such as flattening the image by reshaping it to a single row or column, extracting features using CNNs, or applying dimensionality reduction techniques like PCA. <|end_of_text|>
<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

   

,answer-2,Llama_answers
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio..."
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove..."
...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed..."
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to..."


In [235]:
D1_test_scores_A2 = D1_Test_llama_with_Unsloth_df_A2.copy()

In [236]:
D1_test_scores_A2['rouge_scores'] = D1_test_scores_A2.apply(lambda row: calculate_rouge(row['answer-2'], row['Llama_answers']), axis=1)

In [237]:
D1_test_scores_A2 = pd.concat([D1_test_scores_A2, pd.json_normalize(D1_test_scores_A2['rouge_scores'])], axis=1)

In [238]:
D1_test_scores_A2

,answer-2,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...,"{'rouge1': 0.42424242424242425, 'rouge2': 0.18...",0.424242,0.187500,0.272727,0.272727
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.5882352941176471, 'rouge2': 0.448...",0.588235,0.448980,0.549020,0.549020
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...,"{'rouge1': 0.0975609756097561, 'rouge2': 0.0, ...",0.097561,0.000000,0.097561,0.097561
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...,"{'rouge1': 0.22222222222222224, 'rouge2': 0.09...",0.222222,0.093023,0.222222,0.222222
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove...","{'rouge1': 0.20408163265306123, 'rouge2': 0.0,...",0.204082,0.000000,0.204082,0.204082
...,...,...,...,...,...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...,"{'rouge1': 0.393939393939394, 'rouge2': 0.0625...",0.393939,0.062500,0.212121,0.212121
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed...","{'rouge1': 0.2352941176470588, 'rouge2': 0.0, ...",0.235294,0.000000,0.176471,0.176471
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...,"{'rouge1': 0.375, 'rouge2': 0.0, 'rougeL': 0.3...",0.375000,0.000000,0.312500,0.312500
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to...","{'rouge1': 0.18181818181818182, 'rouge2': 0.0,...",0.181818,0.000000,0.109091,0.109091


In [239]:
D1_test_scores_A2['bleu_score'] = D1_test_scores_A2.apply(lambda row: calculate_bleu(row['answer-2'], row['Llama_answers']), axis=1)


D1_test_scores_A2['meteor_score'] = D1_test_scores_A2.apply(lambda row: calculate_meteor(row['answer-2'], row['Llama_answers']), axis=1)

In [240]:
D1_test_scores_A2

,answer-2,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...,"{'rouge1': 0.42424242424242425, 'rouge2': 0.18...",0.424242,0.187500,0.272727,0.272727,0.852245,0.374150
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.5882352941176471, 'rouge2': 0.448...",0.588235,0.448980,0.549020,0.549020,1.506189,0.371444
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...,"{'rouge1': 0.0975609756097561, 'rouge2': 0.0, ...",0.097561,0.000000,0.097561,0.097561,0.000000,0.054348
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...,"{'rouge1': 0.22222222222222224, 'rouge2': 0.09...",0.222222,0.093023,0.222222,0.222222,0.000000,0.144788
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove...","{'rouge1': 0.20408163265306123, 'rouge2': 0.0,...",0.204082,0.000000,0.204082,0.204082,2.627962,0.074184
...,...,...,...,...,...,...,...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...,"{'rouge1': 0.393939393939394, 'rouge2': 0.0625...",0.393939,0.062500,0.212121,0.212121,0.918708,0.264684
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed...","{'rouge1': 0.2352941176470588, 'rouge2': 0.0, ...",0.235294,0.000000,0.176471,0.176471,0.000000,0.091743
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...,"{'rouge1': 0.375, 'rouge2': 0.0, 'rougeL': 0.3...",0.375000,0.000000,0.312500,0.312500,0.000000,0.198864
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to...","{'rouge1': 0.18181818181818182, 'rouge2': 0.0,...",0.181818,0.000000,0.109091,0.109091,1.335353,0.096463


In [241]:
A2_average_scores = {
    'average_rouge1': D1_test_scores_A2['rouge1'].mean(),
    'average_rouge2': D1_test_scores_A2['rouge2'].mean(),
    'average_rougeL': D1_test_scores_A2['rougeL'].mean(),
    'average_rougeLsum': D1_test_scores_A2['rougeLsum'].mean(),
    'average_bleu_score': D1_test_scores_A2['bleu_score'].mean(),
    'average_meteor_score': D1_test_scores_A2['meteor_score'].mean(),
}

In [242]:
print("Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:\n")
for metric, score in A2_average_scores.items():
    print(f"{metric}: {score:.4f}")


Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:

average_rouge1: 0.3766
average_rouge2: 0.1439
average_rougeL: 0.2885
average_rougeLsum: 0.2918
average_bleu_score: 0.6694
average_meteor_score: 0.2329


In [245]:
# make a df with A2_average_scores and average_scores

# Create a DataFrame from the dictionaries
D1_Test_Scores = pd.DataFrame([average_scores, A2_average_scores], index=['Llama3-8b_model Vs.Answer-1', 'Llama3-8b_model Vs.Answer-2'])

# Transpose the DataFrame for better readability
D1_Test_Scores = D1_Test_Scores.T

D1_Test_Scores


,Llama3-8b_model Vs.Answer-1,Llama3-8b_model Vs.Answer-2
average_rouge1,0.400487,0.376601
average_rouge2,0.177318,0.143933
average_rougeL,0.325214,0.288521
average_rougeLsum,0.328088,0.291801
average_bleu_score,0.693772,0.669431
average_meteor_score,0.268180,0.232883
